In [384]:
import pandas as pd
from pandas.io.json import json_normalize
import re
import json 
import ast
tabla = pd.read_csv('../data/results/ForbesBillionaires2018_add_info.csv', sep=',', index_col=0)

In [385]:
tabla = tabla[['Name','FinancialAssets']].dropna()
tabla

,Name,FinancialAssets
Position,,
1,Jeff Bezos,"[{'exchange': 'NASDAQ', 'ticker': 'AMZN-US', '..."
2,Bill Gates,"[{'exchange': 'NASDAQ', 'ticker': 'ACGL-US', '..."
3,Warren Buffett,"[{'exchange': 'NYSE', 'ticker': 'BRK.A-US', 'c..."
5,Mark Zuckerberg,"[{'exchange': 'NASDAQ', 'ticker': 'FB-US', 'co..."
6,Amancio Ortega,"[{'exchange': 'MADRID', 'ticker': 'ITX-ES', 'c..."
...,...,...
2193,Robert Toll,"[{'exchange': 'NYSE', 'ticker': 'TOL-US', 'com..."
2194,Surin Upatkoon,"[{'exchange': 'MALAYSIA', 'ticker': '3859-MY',..."
2199,Sandy Weill,"[{'exchange': 'NYSE', 'ticker': 'C-US', 'compa..."


In [386]:
Ls =[]

def New_DF(Name,FinancialAssets):
    test_string = '"'+FinancialAssets[1:-1]+'"'
    try: 
        res = json.loads(test_string)
        result_dict = ast.literal_eval(res)
        if type(result_dict) == dict:
            #Ls = [v for k, v in i.items() if k=='exchange']
            for k, v in i.items():
                if k=='exchange':
                    exchange = v
                elif k=='companyName':
                    Ls.append(Name + " ~ " + v + " ~ " + exchange)
        else:
            #Ls = [v for i in result_dict for k, v in i.items() if k=='exchange']
            for i in result_dict:
                for k, v in i.items():
                    if k=='exchange':
                        exchange = v
                    elif k=='companyName':
                        Ls.append(Name + " ~ " + v + " ~ " + exchange)
    except:
        return FinancialAssets
    return list(set(Ls))
        

In [387]:
tabla.apply(lambda row: New_DF(row.Name, row.FinancialAssets), axis=1)

Position
1       [{'exchange': 'NASDAQ', 'ticker': 'AMZN-US', '...
2       [Bill Gates ~ Canadian National Railway Co. ~ ...
3       [Bill Gates ~ Canadian National Railway Co. ~ ...
5       [{'exchange': 'NASDAQ', 'ticker': 'FB-US', 'co...
6       [{'exchange': 'MADRID', 'ticker': 'ITX-ES', 'c...
                              ...                        
2193    [Yupa Chiaravanond ~ Charoen Pokphand Foods ~ ...
2194    [Yupa Chiaravanond ~ Charoen Pokphand Foods ~ ...
2199    [{'exchange': 'NYSE', 'ticker': 'C-US', 'compa...
2206    [Yupa Chiaravanond ~ Charoen Pokphand Foods ~ ...
2207    [{'exchange': 'SHANGHAI', 'ticker': '603228-CN...
Length: 1067, dtype: object

In [388]:
Ls=[]

In [389]:
tabla['masdatos']=tabla.apply(lambda row: New_DF(row.Name, row.FinancialAssets), axis=1)

In [390]:
tabla.to_csv('../data/processed/ForbesBillionairesStokes.csv', index=False)

In [391]:
Ls2 = [i.split(' ~ ') for i in Ls]

df1 = pd.DataFrame(Ls2)
df1.drop_duplicates()
df1.rename(columns = {0:'Name',1:'CompanyName',2:'Exchange'}, inplace = True)
df1

,Name,CompanyName,Exchange
0,Bill Gates,Arch Capital Group,NASDAQ
1,Bill Gates,Arcos Dorados Holdings Inc,NYSE
2,Bill Gates,AutoNation Inc.,NYSE
3,Bill Gates,Berkshire Hathaway Inc. (Cl A),NYSE
4,Bill Gates,Canadian National Railway Co.,NYSE
...,...,...,...
2620,Robert Toll,Toll Brothers Inc.,NYSE
2621,Surin Upatkoon,Magnum Berhad,MALAYSIA
2622,Surin Upatkoon,MPHB Capital Sdn. Bhd,MALAYSIA
2623,Zhu Xingming,Shenzhen Inovance Technology Co Ltd,SHENZHEN


In [392]:
report7 = df1[['Name','CompanyName','Exchange']].drop_duplicates()
report7 = pd.DataFrame(report7.groupby(['CompanyName','Exchange'])['Name'].count().sort_values(ascending=False))
report7.rename(columns = {'Name':'Count'}, inplace = True)
report7['Percentage']=round(report7.Count/sum(report7.Count)*100,1)
report7 = report7.reset_index().head(3)
report7.index += 1
report7 = report7[['CompanyName','Exchange']]
report7

,CompanyName,Exchange
1,Facebook,NASDAQ
2,Alphabet,NASDAQ
3,Google Inc. (Cl C),NASDAQ


In [393]:
report8 = df1[['Exchange','Name']].drop_duplicates()
report8 = pd.DataFrame(report8.groupby('Exchange')['Name'].count().sort_values(ascending=False))
report8.rename(columns = {'Name':'Count'}, inplace = True)
report8['Percentage']=round(report8.Count/sum(report8.Count)*100,1)
report8 = report8.reset_index().head(3)
report8.index += 1
report8

,Exchange,Count,Percentage
1,NYSE,138,14.2
2,NASDAQ,116,11.9
3,HONG KONG,85,8.7


In [394]:
report9 = df1[['Name','CompanyName']].drop_duplicates()
report9 = pd.DataFrame(report9.groupby('Name')['CompanyName'].count().sort_values(ascending=False))
report9.rename(columns = {'CompanyName':'Count'}, inplace = True)
report9 = report9.reset_index().head(3)
report9.index += 1
report9

,Name,Count
1,Rakesh Jhunjhunwala,37
2,Arturo Moreno,35
3,John Malone,33
